In [1]:
import cv2 as cv
import numpy as np
from matplotlib import pyplot as plt
import os

In [2]:
def getNewSize(yxDiff):
    # Take yxDiff as input, return how much to crop from each side
    side1 = abs(yxDiff)/2
    if side1.is_integer():
        side2 = side1
    else:
        side2 = int(side1) + 1
    return int(side1), int(side2)

In [3]:
def cropImage(img):
    # Cropstatus is used to know if an image needs cropping
    cropStatus = 0
    # Get aspect ratio
    yxDiff = img.shape[1] - img.shape[0]
    
    if yxDiff == 0:
        # Do not crop
        newImage = img
    elif yxDiff > 0:
        # Crop in x-direction, eaven on each side
        cropStatus = 1
        side1, side2 = getNewSize(yxDiff)
        newImage = img[:, side1:-side2, :]
    elif yxDiff < 0:
        # Crop in y-direction, eaven on each side
        cropStatus = 1
        side1, side2 = getNewSize(yxDiff)
        newImage = img[side1:-side2, :, :]
    return newImage, cropStatus

In [6]:
'''
Script is running on the given folders, benign and malignant. 
Loops over each file and transforms them to 224 x 224 images. It keeps
the new image centered compared to before.
The new image is stored in the folders with the same name + _224_224.
A log-file is created to know which files needed cropping, this can be 
used later when manually checking files.
'''

# Log which files has been altered in aspect ratio, manual check
logFile = open('logging.csv', 'a')

for directory in ['benign', 'malignant']:
    for image in os.listdir(directory):
        img = cv.imread(directory+'/'+image, -1)
        croppedImage, cropStatus = cropImage(img)
        if cropStatus == 1:
            logFile.write(image + ",")
        resizedImage = cv.resize(croppedImage, (224,224))

        # Write file to disk
        if not cv.imwrite(directory+"_224_224/"+image, resizedImage):
            print("Could not write file!", image)

logFile.close()